In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import os
from tqdm import tqdm
from pylab import rcParams
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from torch import nn, optim

In [1]:
df = pd.read_csv("~pdata.csv")

NameError: name 'pd' is not defined

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df = df.loc[0:118]
df = df[['날짜', '정책', '정책지수', '누적정책지수', '서울', '서울 비율', '용산구', '광진구', '서대문구',
       '영등포구', '서초구', '소비자물가지수', '한국은행기준금리', '원달러환율', '가계대출', '서울시부동산심리지수',
       '서울실거래지수', '도심권실거래지수', '동북권실거래지수', '서북권실거래지수', '서남권실거래지수', '동남권실거래지수']]
df.columns = ["date","p1", "p2","p3","Seoul","Seoul_ratio","Yongsan-gu","Gwangjin-gu","Seodaemun-gu",
              "Yeongdeungpo-gu","Seocho-gu", "CPI","BR","ER","HL", "CSI",
              "SP", "MP", "ENP", "WNP", "WSP", "ESP"]

In [ ]:
df.head()

In [ ]:
#df["ER"]=df["ER"].str.replace(',', '').astype('float')
#df["HL"]=df["HL"].str.replace(',', '').astype('float')

In [ ]:
df = df[["date","p1","p2","p3","Seoul","Yongsan-gu","Gwangjin-gu","Seodaemun-gu",
              "Yeongdeungpo-gu","Seocho-gu","CPI","BR","ER","HL", "CSI"]]

In [ ]:
data = df[["Yongsan-gu","Gwangjin-gu","Seodaemun-gu",
              "Yeongdeungpo-gu","Seocho-gu","p2","CPI","BR","ER","HL", "CSI"]]

# Split feature and label

In [ ]:
df1 = df[["p2","CPI","BR","ER","HL","CSI"]]
df2 = df[["Yongsan-gu","p2","CPI","BR","ER","HL","CSI"]]
df3 = df[["Gwangjin-gu", "p2","CPI","BR","ER","HL","CSI"]]
df4 = df[["Seodaemun-gu","p2","CPI","BR","ER","HL","CSI"]]
df5 = df[["Yeongdeungpo-gu", "p2","CPI","BR","ER","HL","CSI"]]
df6 = df[["Seocho-gu", "p2", "CPI","BR","ER","HL","CSI"]]

# Yongsan-gu

In [ ]:
from sklearn.preprocessing import MinMaxScaler

label=df2

#feature.sort_index(ascending=False).reset_index(drop=True)
#label.sort_index(ascending=False).reset_index(drop=True)

feature_scaler = MinMaxScaler()
label_scaler = MinMaxScaler()

feature = feature_scaler.fit_transform(df1)
feature = pd.DataFrame(feature)

label = label_scaler.fit_transform(label)
label = pd.DataFrame(label)

In [ ]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score

kfold=KFold(10)
cvscores = []

In [ ]:
import os
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_loss', patience=5)

model_path = 'model'
filename = os.path.join(model_path, 'tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')


In [ ]:
import random
seed = 1234
random.seed(seed)

from keras.models import Sequential
from keras.layers import Activation,Dense,Flatten,LSTM,Dropout,Conv1D, MaxPooling1D, TimeDistributed

for train, test in kfold.split(feature,label):
    
    train_feature, test_feature = feature[0:106], feature[93:106]
    train_label, test_label = label[0:106], label[93:106]
    pred_feature, pred_label = feature[105:118], label[105:118]

    train_feature, train_label = make_dataset(train_feature,train_label, 6)
    test_feature, test_label = make_dataset(test_feature, test_label, 6)
    pred_feature, pred_label = make_dataset(pred_feature, pred_label, 6)
    
    x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)
    
    subsequences = 2
    timesteps = x_train.shape[1]//subsequences
    X_train_series_sub = x_train.reshape((x_train.shape[0], subsequences, timesteps, 6)) 
    X_valid_series_sub = x_valid.reshape((x_valid.shape[0], subsequences, timesteps, 6))
    test_feature_series_sub = test_feature.reshape((test_feature.shape[0], subsequences, timesteps, 6))
    pred_feature_series_sub = pred_feature.reshape((pred_feature.shape[0], subsequences, timesteps, 6))
    
    output_size=6 
    neurons=100 
    loss="mean_squared_error" 
    
    model_cnn_lstm = Sequential()
    model_cnn_lstm.add(TimeDistributed(Conv1D(filters=128, kernel_size=2, 
                                          activation='relu'),
                                   input_shape=(None, X_train_series_sub.shape[2],
                                                X_train_series_sub.shape[3])))
    model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model_cnn_lstm.add(TimeDistributed(Dropout((0.5))))
    model_cnn_lstm.add(TimeDistributed(Flatten()))
    model_cnn_lstm.add(LSTM(100, activation='relu'))
    model_cnn_lstm.add(Dropout(0.25))
    model_cnn_lstm.add(Dense(150))
    model_cnn_lstm.add(Dense(1))
    model_cnn_lstm.compile(loss=loss, optimizer="adam", metrics=["mse"])
    model_cnn_lstm.fit(X_train_series_sub, y_train,
                    epochs=50, 
                    verbose=1, 
                    validation_data=(X_valid_series_sub, y_valid), 
                    callbacks=[early_stop, checkpoint])
    scores = model_cnn_lstm.evaluate(test_feature_series_sub, test_label, verbose=0)
    cvscores = (scores * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))        

In [ ]:
test_pred = model_cnn_lstm.predict(test_feature_series_sub)

In [ ]:
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))
print('RMSE : ', RMSE(test_label, test_pred)) 

In [ ]:
test_pred = label_scaler.inverse_transform(test_pred)
test_label = label_scaler.inverse_transform(test_label)

In [ ]:
pred = model_cnn_lstm.predict(pred_feature_series_sub)
true = pred_label

In [ ]:
RMSE(true, pred)

In [ ]:
pred = label_scaler.inverse_transform(pred)
pred

In [ ]:
true = label2.tail(6)
true = np.array(true)
true

In [ ]:
plt.plot(true, label = 'actual')
plt.plot(pred, label = 'prediction')
plt.legend()
plt.show()

In [ ]:
#a=label2
#b=pd.DataFrame(pred, columns = ["Yongsan-gu"])
#mod_df = a.append(b.loc[:], ignore_index=True)
#mod_df

# Gwangjin-gu

In [ ]:
from sklearn.preprocessing import MinMaxScaler

label=label3

#feature.sort_index(ascending=False).reset_index(drop=True)
#label.sort_index(ascending=False).reset_index(drop=True)

feature_scaler = MinMaxScaler()
label_scaler = MinMaxScaler()

feature = feature_scaler.fit_transform(feature)
feature = pd.DataFrame(feature)

label = label_scaler.fit_transform(label)
label = pd.DataFrame(label)

In [ ]:
import random
seed = 1234
random.seed(seed)

from keras.models import Sequential
from keras.layers import Activation,Dense,Flatten,LSTM,Dropout,Conv1D, MaxPooling1D, TimeDistributed

for train, test in kfold.split(feature,label):
    
    train_feature, test_feature = feature[0:106], feature[93:106]
    train_label, test_label = label[0:106], label[93:106]
    pred_feature, pred_label = feature[105:118], label[105:118]

    train_feature, train_label = make_dataset(train_feature,train_label, 6)
    test_feature, test_label = make_dataset(test_feature, test_label, 6)
    pred_feature, pred_label = make_dataset(pred_feature, pred_label, 6)
    
    x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)
    
    subsequences = 2
    timesteps = x_train.shape[1]//subsequences
    X_train_series_sub = x_train.reshape((x_train.shape[0], subsequences, timesteps, 6)) 
    X_valid_series_sub = x_valid.reshape((x_valid.shape[0], subsequences, timesteps, 6))
    test_feature_series_sub = test_feature.reshape((test_feature.shape[0], subsequences, timesteps, 6))
    pred_feature_series_sub = pred_feature.reshape((pred_feature.shape[0], subsequences, timesteps, 6))
    
    output_size=6 
    neurons=100 
    loss="mean_squared_error" 
    
    model_cnn_lstm = Sequential()
    model_cnn_lstm.add(TimeDistributed(Conv1D(filters=128, kernel_size=2, 
                                          activation='relu'),
                                   input_shape=(None, X_train_series_sub.shape[2],
                                                X_train_series_sub.shape[3])))
    model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model_cnn_lstm.add(TimeDistributed(Dropout((0.5))))
    model_cnn_lstm.add(TimeDistributed(Flatten()))
    model_cnn_lstm.add(LSTM(100, activation='relu'))
    model_cnn_lstm.add(Dropout(0.25))
    model_cnn_lstm.add(Dense(150))
    model_cnn_lstm.add(Dense(1))
    model_cnn_lstm.compile(loss=loss, optimizer="adam", metrics=["mse"])
    model_cnn_lstm.fit(X_train_series_sub, y_train,
                    epochs=50, 
                    verbose=1, 
                    validation_data=(X_valid_series_sub, y_valid), 
                    callbacks=[early_stop, checkpoint])
    scores = model_cnn_lstm.evaluate(test_feature_series_sub, test_label, verbose=0)
    cvscores = (scores * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores))) 

In [ ]:
test_pred = model_cnn_lstm.predict(test_feature_series_sub)

In [ ]:
RMSE(test_label, test_pred)

In [ ]:
test_pred = label_scaler.inverse_transform(test_pred)
test_label = label_scaler.inverse_transform(test_label)

In [ ]:
pred = model_cnn_lstm.predict(pred_feature_series_sub)
true = pred_label

In [ ]:
RMSE(true, pred)

In [ ]:
pred = label_scaler.inverse_transform(pred)
pred

In [ ]:
true = label3.tail(6)
true = np.array(true)
true

In [ ]:
plt.plot(true, label = 'actual')
plt.plot(pred, label = 'prediction')
plt.legend()
plt.show()

# Seodaemun-gu

In [ ]:
from sklearn.preprocessing import MinMaxScaler

label=label4

#feature.sort_index(ascending=False).reset_index(drop=True)
#label.sort_index(ascending=False).reset_index(drop=True)

feature_scaler = MinMaxScaler()
label_scaler = MinMaxScaler()

feature = feature_scaler.fit_transform(feature)
feature = pd.DataFrame(feature)

label = label_scaler.fit_transform(label)
label = pd.DataFrame(label)

In [ ]:
import random
seed = 1234
random.seed(seed)

from keras.models import Sequential
from keras.layers import Activation,Dense,Flatten,LSTM,Dropout,Conv1D, MaxPooling1D, TimeDistributed

for train, test in kfold.split(feature,label):
    
    train_feature, test_feature = feature[0:106], feature[93:106]
    train_label, test_label = label[0:106], label[93:106]
    pred_feature, pred_label = feature[105:118], label[105:118]

    train_feature, train_label = make_dataset(train_feature,train_label, 6)
    test_feature, test_label = make_dataset(test_feature, test_label, 6)
    pred_feature, pred_label = make_dataset(pred_feature, pred_label, 6)
    
    x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)
    
    subsequences = 2
    timesteps = x_train.shape[1]//subsequences
    X_train_series_sub = x_train.reshape((x_train.shape[0], subsequences, timesteps, 6)) 
    X_valid_series_sub = x_valid.reshape((x_valid.shape[0], subsequences, timesteps, 6))
    test_feature_series_sub = test_feature.reshape((test_feature.shape[0], subsequences, timesteps, 6))
    pred_feature_series_sub = pred_feature.reshape((pred_feature.shape[0], subsequences, timesteps, 6))
    
    output_size=6 
    neurons=100 
    loss="mean_squared_error" 
    
    model_cnn_lstm = Sequential()
    model_cnn_lstm.add(TimeDistributed(Conv1D(filters=128, kernel_size=2, 
                                          activation='relu'),
                                   input_shape=(None, X_train_series_sub.shape[2],
                                                X_train_series_sub.shape[3])))
    model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model_cnn_lstm.add(TimeDistributed(Dropout((0.5))))
    model_cnn_lstm.add(TimeDistributed(Flatten()))
    model_cnn_lstm.add(LSTM(100, activation='relu'))
    model_cnn_lstm.add(Dropout(0.25))
    model_cnn_lstm.add(Dense(150))
    model_cnn_lstm.add(Dense(1))
    model_cnn_lstm.compile(loss=loss, optimizer="adam", metrics=["mse"])
    model_cnn_lstm.fit(X_train_series_sub, y_train,
                    epochs=50, 
                    verbose=1, 
                    validation_data=(X_valid_series_sub, y_valid), 
                    callbacks=[early_stop, checkpoint])
    scores = model_cnn_lstm.evaluate(test_feature_series_sub, test_label, verbose=0)
    cvscores = (scores * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores))) 

In [ ]:
test_pred = model_cnn_lstm.predict(test_feature_series_sub)

In [ ]:
RMSE(test_label, test_pred)

In [ ]:
test_pred = label_scaler.inverse_transform(test_pred)
test_label = label_scaler.inverse_transform(test_label)

In [ ]:
pred = model_cnn_lstm.predict(pred_feature_series_sub)
true = pred_label

In [ ]:
RMSE(true, pred)

In [ ]:
pred = label_scaler.inverse_transform(pred)
pred

In [ ]:
true = label4.tail(6)
true = np.array(true)
true

In [ ]:
plt.plot(true, label = 'actual')
plt.plot(pred, label = 'prediction')
plt.legend()
plt.show()

# Yeongdeungpo-gu

In [ ]:
from sklearn.preprocessing import MinMaxScaler

label=label5

#feature.sort_index(ascending=False).reset_index(drop=True)
#label.sort_index(ascending=False).reset_index(drop=True)

feature_scaler = MinMaxScaler()
label_scaler = MinMaxScaler()

feature = feature_scaler.fit_transform(feature)
feature = pd.DataFrame(feature)

label = label_scaler.fit_transform(label)
label = pd.DataFrame(label)

In [ ]:
import random
seed = 1234
random.seed(seed)

from keras.models import Sequential
from keras.layers import Activation,Dense,Flatten,LSTM,Dropout,Conv1D, MaxPooling1D, TimeDistributed

for train, test in kfold.split(feature,label):
    
    train_feature, test_feature = feature[0:106], feature[93:106]
    train_label, test_label = label[0:106], label[93:106]
    pred_feature, pred_label = feature[105:118], label[105:118]

    train_feature, train_label = make_dataset(train_feature,train_label, 6)
    test_feature, test_label = make_dataset(test_feature, test_label, 6)
    pred_feature, pred_label = make_dataset(pred_feature, pred_label, 6)
    
    x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)
    
    subsequences = 2
    timesteps = x_train.shape[1]//subsequences
    X_train_series_sub = x_train.reshape((x_train.shape[0], subsequences, timesteps, 6)) 
    X_valid_series_sub = x_valid.reshape((x_valid.shape[0], subsequences, timesteps, 6))
    test_feature_series_sub = test_feature.reshape((test_feature.shape[0], subsequences, timesteps, 6))
    pred_feature_series_sub = pred_feature.reshape((pred_feature.shape[0], subsequences, timesteps, 6))
    
    output_size=6 
    neurons=100 
    loss="mean_squared_error" 
    
    model_cnn_lstm = Sequential()
    model_cnn_lstm.add(TimeDistributed(Conv1D(filters=128, kernel_size=2, 
                                          activation='relu'),
                                   input_shape=(None, X_train_series_sub.shape[2],
                                                X_train_series_sub.shape[3])))
    model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model_cnn_lstm.add(TimeDistributed(Dropout((0.5))))
    model_cnn_lstm.add(TimeDistributed(Flatten()))
    model_cnn_lstm.add(LSTM(100, activation='relu'))
    model_cnn_lstm.add(Dropout(0.25))
    model_cnn_lstm.add(Dense(150))
    model_cnn_lstm.add(Dense(1))
    model_cnn_lstm.compile(loss=loss, optimizer="adam", metrics=["mse"])
    model_cnn_lstm.fit(X_train_series_sub, y_train,
                    epochs=50, 
                    verbose=1, 
                    validation_data=(X_valid_series_sub, y_valid), 
                    callbacks=[early_stop, checkpoint])
    scores = model_cnn_lstm.evaluate(test_feature_series_sub, test_label, verbose=0)
    cvscores = (scores * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores))) 

In [ ]:
test_pred = model_cnn_lstm.predict(test_feature_series_sub)

In [ ]:
RMSE(test_label, test_pred)

In [ ]:
test_pred = label_scaler.inverse_transform(test_pred)
test_label = label_scaler.inverse_transform(test_label)

In [ ]:
pred = model_cnn_lstm.predict(pred_feature_series_sub)
true = pred_label

In [ ]:
RMSE(true, pred)

In [ ]:
pred = label_scaler.inverse_transform(pred)
pred

In [ ]:
true = label5.tail(6)
true = np.array(true)
true

In [ ]:
plt.plot(true, label = 'actual')
plt.plot(pred, label = 'prediction')
plt.legend()
plt.show()

# Seocho-gu

In [ ]:
from sklearn.preprocessing import MinMaxScaler

label=label6

#feature.sort_index(ascending=False).reset_index(drop=True)
#label.sort_index(ascending=False).reset_index(drop=True)

feature_scaler = MinMaxScaler()
label_scaler = MinMaxScaler()

feature = feature_scaler.fit_transform(feature)
feature = pd.DataFrame(feature)

label = label_scaler.fit_transform(label)
label = pd.DataFrame(label)

In [ ]:
import random
seed = 1234
random.seed(seed)

from keras.models import Sequential
from keras.layers import Activation,Dense,Flatten,LSTM,Dropout,Conv1D, MaxPooling1D, TimeDistributed

for train, test in kfold.split(feature,label):
    
    train_feature, test_feature = feature[0:106], feature[93:106]
    train_label, test_label = label[0:106], label[93:106]
    pred_feature, pred_label = feature[105:118], label[105:118]

    train_feature, train_label = make_dataset(train_feature,train_label, 6)
    test_feature, test_label = make_dataset(test_feature, test_label, 6)
    pred_feature, pred_label = make_dataset(pred_feature, pred_label, 6)
    
    x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)
    
    subsequences = 2
    timesteps = x_train.shape[1]//subsequences
    X_train_series_sub = x_train.reshape((x_train.shape[0], subsequences, timesteps, 6)) 
    X_valid_series_sub = x_valid.reshape((x_valid.shape[0], subsequences, timesteps, 6))
    test_feature_series_sub = test_feature.reshape((test_feature.shape[0], subsequences, timesteps, 6))
    pred_feature_series_sub = pred_feature.reshape((pred_feature.shape[0], subsequences, timesteps, 6))
    
    output_size=6 
    neurons=100 
    loss="mean_squared_error" 
    
    model_cnn_lstm = Sequential()
    model_cnn_lstm.add(TimeDistributed(Conv1D(filters=128, kernel_size=2, 
                                          activation='relu'),
                                   input_shape=(None, X_train_series_sub.shape[2],
                                                X_train_series_sub.shape[3])))
    model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model_cnn_lstm.add(TimeDistributed(Dropout((0.5))))
    model_cnn_lstm.add(TimeDistributed(Flatten()))
    model_cnn_lstm.add(LSTM(100, activation='relu'))
    model_cnn_lstm.add(Dropout(0.25))
    model_cnn_lstm.add(Dense(150))
    model_cnn_lstm.add(Dense(1))
    model_cnn_lstm.compile(loss=loss, optimizer="adam", metrics=["mse"])
    model_cnn_lstm.fit(X_train_series_sub, y_train,
                    epochs=50, 
                    verbose=1, 
                    validation_data=(X_valid_series_sub, y_valid), 
                    callbacks=[early_stop, checkpoint])
    scores = model_cnn_lstm.evaluate(test_feature_series_sub, test_label, verbose=0)
    cvscores = (scores * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores))) 

In [ ]:
test_pred = model_cnn_lstm.predict(test_feature_series_sub)

In [ ]:
RMSE(test_label, test_pred)

In [ ]:
test_pred = label_scaler.inverse_transform(test_pred)
test_label = label_scaler.inverse_transform(test_label)

In [ ]:
pred = model_cnn_lstm.predict(pred_feature_series_sub)
true = pred_label

In [ ]:
RMSE(true, pred)

In [ ]:
pred = label_scaler.inverse_transform(pred)
pred

In [ ]:
true = label6.tail(6)
true = np.array(true)
true

In [ ]:
plt.plot(true, label = 'actual')
plt.plot(pred, label = 'prediction')
plt.legend()
plt.show()